In [12]:
import statsmodels.formula.api as smf
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [14]:
data = pd.read_csv('../data/dem_with_dummies_renamed.csv')

In [15]:
data = data.drop("Candidate", axis=1)
data = data.drop("State", axis=1)
data = data.drop("Primary_Vote_Percentage", axis=1)

In [16]:
train_features, test_features, train_outcome, test_outcome = train_test_split(
    data.drop('Primary_Status_Advanced', axis=1),
    data['Primary_Status_Advanced'],
    test_size = 0.3,
    random_state = 11
)

In [17]:
train_features['Primary_Status_Advanced'] = (train_outcome).to_frame()

In [ ]:
model = forward_selected(train_features, 'Primary_Status_Advanced')